In [5]:
%matplotlib inline

Created on Thu Jun 16 11:19:17 2022
developed by Felix Panitz* and Peter Stange*
* at Chair of Building Energy Systems and Heat Supply, 
  Technische Universität Dresden

In [6]:
import numpy as np
import datetime
from flixOpt.flixStructure import *
from flixOpt.flixComps    import *

# some timeseries ##

In [7]:
Q_th_Last = [30., 0., 20.] # kW; thermal load profile in
aTimeSeries = datetime.datetime(2020, 1,1) +  np.arange(len(Q_th_Last)) * datetime.timedelta(hours=1) # creating timeseries
aTimeSeries = aTimeSeries.astype('datetime64') # needed format for timeseries in flixOpt

print('#######################################################################')
print('################### start of modeling #################################')

#######################################################################
################### start of modeling #################################


## Bus-Definition: ##
define buses for the 3 used media:

In [8]:
Strom = Bus('el', 'Strom') # balancing node/bus of electricity
Fernwaerme = Bus('heat', 'Fernwärme') # balancing node/bus of heat
Gas = Bus('fuel', 'Gas') # balancing node/bus of gas

Name 'Fernwärme' ist nicht Attributnamen-konform und wird zu 'Fernwaerme' geändert


## Effect-Definition: ##

In [9]:
costs = Effect('costs','€','Kosten',  # name, unit, description
                    isStandard = True, # standard effect --> shorter input possible (without effect as a key)
                    isObjective = True) # defining costs as objective of optimiziation

## Component-Definition: ##

In [10]:
aBoiler = Boiler('Boiler', eta = 0.5, # name, efficiency factor
                  # defining the output-flow = thermal -flow
                  Q_th = Flow(label = 'Q_th', # name of flow
                               bus = Fernwaerme, # define, where flow is linked to (here: Fernwaerme-Bus)
                               nominal_val = 50, # kW; nominal_size of boiler
                               ),    
                  # defining the input-flow = fuel-flow
                  Q_fu = Flow(label = 'Q_fu', # name of flow
                               bus = Gas) # define, where flow is linked to (here: Gas-Bus)
                  )

# sink of heat load:
aWaermeLast = Sink('Wärmelast',
                    # defining input-flow:
                    sink   = Flow('Q_th_Last', # name
                                   bus = Fernwaerme, # linked to bus "Fernwaerme"
                                   nominal_val = 1, # nominal_value
                                   val_rel = Q_th_Last)) # fixed profile
                                   # relative fixed values (timeseries) of the flow
                                   # value = val_rel * nominal_val
    
# source of gas:
aGasTarif = Source('Gastarif' ,
                    # defining output-flow:
                    source = Flow('Q_Gas', # name
                                   bus = Gas, # linked to bus "Gas"
                                   nominal_val = 1000, # nominal size, i.e. 1000 kW maximum
                                   # defining effect-shares. 
                                   #    Here not only "costs", but also CO2-emissions:
                                   costsPerFlowHour= 0.04)) # 0.04 €/kWh


Name 'Wärmelast' ist nicht Attributnamen-konform und wird zu 'Waermelast' geändert


## Build energysystem - Registering of all elements ##

In [11]:
system = System(aTimeSeries, dt_last=None) # creating system, (duration of last timestep is like the one before)
system.addEffects(costs) # adding defined effects
system.addComponents(aBoiler, aWaermeLast, aGasTarif) # adding components

# choose used timeindexe:
time_indices = None # all timeindexe are used

# ## modeling the system ##

# 1. create a Calculation 
aCalc = Calculation('Sim1', # name of calculation
                     system, # energysystem to calculate
                     'pyomo', # optimization modeling language (only "pyomo" implemented, yet)
                     time_indices) # used time steps

# 2. modeling:
aCalc.doModelingAsOneSegment() # mathematic modeling of system

# 3. (optional) print Model-Characteristics:
system.printModel() # string-output:network structure of model
system.printVariables() # string output: variables of model
system.printEquations() # string-output: equations of model


# #################
# ## calculation ##

### some Solver-Inputs: ###
displaySolverOutput = True  # ausführlicher Solver-Output.
gapFrac = 0.01 # solver-gap
timelimit = 3600 # seconds until solver abort
# choose the solver, you have installed:
# solver_name = 'glpk' # warning, glpk quickly has numerical problems with binaries (big and epsilon)
# solver_name = 'gurobi'
solver_name = 'cbc'
solverProps = {'gapFrac': gapFrac,
               'timelimit': timelimit,
               'solver': solver_name,
               'displaySolverOutput' : displaySolverOutput,
               }

aCalc.solve(solverProps, # some solver options
            nameSuffix = '_' + solver_name) # nameSuffix for the results
#  results are saved under /results/

# ##### loading results from output-files ######
import flixOpt.flixPostprocessing as flixPost

aCalc_post = flixPost.flix_results(aCalc.nameOfCalc)


Register new effect costs
Register new Component Boiler
Register new Component Waermelast
Register new Component Gastarif
finalize all Elements...
Boiler
Waermelast
Gastarif
Gas
Fernwaerme
globalComp
costs
Q_Gas
Q_fu
Q_th_Last
Q_th
modeling costs
declare_vars_and_eqs Q_fu
declare_vars_and_eqs Q_th
declare_vars_and_eqs Q_th_Last
declare_vars_and_eqs Q_Gas

##############################################################
########## Short String Description of Energysystem ##########

buses:
  Fernwaerme:
    In-Flows:
    - Q_th:
        comp: Boiler
    Out-Flows:
    - Q_th_Last:
        comp: Waermelast
    class: Bus
  Gas:
    In-Flows:
    - Q_Gas:
        comp: Gastarif
    Out-Flows:
    - Q_fu:
        comp: Boiler
    class: Bus
components:
  Boiler:
    In-Flows:
    - Q_fu:
        bus: Gas
    Out-Flows:
    - Q_th:
        bus: Fernwaerme
    class: Boiler
  Gastarif:
    In-Flows: []
    Out-Flows:
    - Q_Gas:
        bus: Gas
    class: Source
  Waermelast:
    In-Flows:
 